<a href="https://colab.research.google.com/github/dvircohen0/NLP/blob/main/semantic_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import random
import textwrap

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
!wget https://raw.githubusercontent.com/AmirYunus/py_sentiment_analysis/9f0a92edc228d6af49f9bbeb66dcc80e8b412e29/electronics/positive.review
!wget https://raw.githubusercontent.com/AmirYunus/py_sentiment_analysis/9f0a92edc228d6af49f9bbeb66dcc80e8b412e29/electronics/negative.review

--2021-05-19 08:57:34--  https://raw.githubusercontent.com/AmirYunus/py_sentiment_analysis/9f0a92edc228d6af49f9bbeb66dcc80e8b412e29/electronics/positive.review
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1105010 (1.1M) [text/plain]
Saving to: ‘positive.review’

positive.review     100%[===================>]   1.05M  --.-KB/s    in 0.04s   

2021-05-19 08:57:34 (23.7 MB/s) - ‘positive.review’ saved [1105010/1105010]

--2021-05-19 08:57:34--  https://raw.githubusercontent.com/AmirYunus/py_sentiment_analysis/9f0a92edc228d6af49f9bbeb66dcc80e8b412e29/electronics/negative.review
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent

In [34]:
positive_reviews=[]
pos_rev =  BeautifulSoup(open('/content/positive.review').read())
for p in pos_rev.findAll('review_text'):
  positive_reviews.append(p.text.strip())

negative_reviews =[]
neg_rev =  BeautifulSoup(open('/content/negative.review').read())
for n in neg_rev.findAll('review_text'):
  negative_reviews.append(n.text.strip())

np.random.shuffle(positive_reviews)
positive_reviews = positive_reviews[:len(negative_reviews)]

In [43]:
pos_review = random.choice(positive_reviews)
neg_review = random.choice(negative_reviews)
print("positive review: \n")
print("\n".join(textwrap.wrap(pos_review,70)) + "\n")
print("negatice review: \n")
print("\n".join(textwrap.wrap(neg_review,70)))

positive review: 

I was in the market for a new inkjet and finally settled on this
printer. The photo output is awesome and it's very fast!  There are
only two things I would comment on that new users might not be aware
of. I was ready to return the printer as I thought it was defective
until I found out what the problems were.  1) When the cartridges dry
out, it will tell you that the cartridges are incompatible. This was
confusing as I did have the correct cartridges in the printer. A
"replace cartridge" message would have been less ambiguous.  2) The
printer has so far appeared "dead" once. When I went to turn it on, it
would not do so. The solution is on HP's website - you must remove the
rear door and do a hard reset. I did this and the printer worked fine
again.  The printer does not come with a USB cable, but those can be
easily bought for $14-20.

negatice review: 

I've had it for one week only and I'm using 3rd pair of Duracell
batteries right now! I was using it for 3-4 hou

In [44]:
def tokenizer(text):
    text = text.lower()
    tokens = nltk.tokenize.word_tokenize(text)
    tokens = [t for t in tokens if len(t) > 2]
    tokens = [Word_lemmatizer.lemmatize(t) for t in tokens]
    tokens = [t for t in tokens if t not in stop_words]
    return tokens

In [45]:
def tokens_to_vector(tokens, label):
    x = np.zeros(len(word_map) + 1)
    for t in tokens:
        i = word_map[t]
        x[i] +=1
    x = x/x.sum()
    x[-1] = label
    return x

In [51]:
Word_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
print("stop words list: \n")
print("\n".join(textwrap.wrap(", ".join(stop_words),70)))

stop word list: 

into, through, weren't, y, him, o, all, her, these, its, yourself,
theirs, before, very, doing, mightn't, aren't, am, a, isn, about,
over, she's, doesn't, shan't, more, while, doesn, couldn't, against,
d, our, after, on, when, hasn't, such, that, needn't, yours, mustn,
isn't, you'd, my, ll, hasn, hadn't, shouldn, ourselves, was, did,
once, too, further, no, hadn, who, in, because, under, those, not,
it's, by, than, yourselves, m, mightn, own, or, just, needn, can,
weren, how, until, wouldn, does, she, that'll, above, down, won't, be,
having, with, being, you've, you'll, ain, is, been, has, haven't, the,
don't, mustn't, their, themselves, i, t, out, there, didn, here,
where, itself, should've, what, will, this, himself, myself, and,
haven, at, couldn, up, ve, re, shouldn't, during, wasn, were, they,
but, between, won, as, most, s, do, ma, then, are, any, off, for, now,
herself, same, ours, it, don, to, which, shan, didn't, he, we, of,
aren, had, other, wouldn't, few, a

In [57]:
word_map = {}
current_index = 0
positive_tokenized = []
negative_tokenized = []

for review in positive_reviews:
    tokens = tokenizer(review)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_map:
            word_map[token] = current_index
            current_index +=1
            
for review in negative_reviews:
    tokens = tokenizer(review)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_map:
            word_map[token] = current_index
            current_index +=1

print("Total number of words in corpus: ",len(word_map))

Total number of words in corpus:  11293


In [58]:
N = len(negative_tokenized) + len(negative_tokenized)
data = np.zeros((N,len(word_map) + 1))
i = 0
for tokens in positive_tokenized:
    xy = tokens_to_vector(tokens, 1)
    data[i,:] = xy
    i +=1

for tokens in negative_tokenized:
    xy = tokens_to_vector(tokens, 0)
    data[i,:] = xy
    i +=1

X = data[:,:-1]
Y = data[:,-1]

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.05, random_state=42)

In [60]:
LR_model = LogisticRegression()
RF_model =  RandomForestClassifier()
AB_model =  AdaBoostClassifier()
NB_model = MultinomialNB()
SVM_model = svm.LinearSVC()


LR_model.fit(X_train, y_train)
RF_model.fit(X_train, y_train)
AB_model.fit(X_train, y_train)
NB_model.fit(X_train, y_train)
SVM_model.fit(X_train, y_train)


print("Logistic Regression classfifcation score: ", LR_model.score(X_test,y_test))
print("Random Forest classfifcation score: ", RF_model.score(X_test,y_test))
print("AdaBoost classfifcation score: ", AB_model.score(X_test,y_test))
print("MultinomialNB classfifcation score: ", NB_model.score(X_test,y_test))
print("SVM classfifcation score: ", SVM_model.score(X_test,y_test))


Logistic Regression classfifcation score:  0.77
Random Forest classfifcation score:  0.85
AdaBoost classfifcation score:  0.7
MultinomialNB classfifcation score:  0.78
SVM classfifcation score:  0.78


In [134]:
treshold = 0.5

for word, index in word_map.items():
    weight = LR_model.coef_[0][index]
    if weight > treshold:
      pos_words.append(word)
    elif weight < -treshold:
      neg_words.append(word)


print("Positive words: \n")
print("\n".join(textwrap.wrap(", ".join(pos_words),70))+"\n")  
print("Negative words: \n")
print("\n".join(textwrap.wrap(", ".join(neg_words),70)))    

Positive words: 

price, 've, ha, love, easy, use, used, lot, memory, small, happy,
recommend, well, good, sound, quality, problem, cable, little, paper,
excellent, great, using, like, speaker, also, perfect, need, best,
fast, pretty, comfortable, highly, price, 've, ha, love, easy, use,
used, lot, memory, small, happy, recommend, well, good, sound,
quality, problem, cable, little, paper, excellent, great, using, like,
speaker, also, perfect, need, best, fast, pretty, comfortable, highly

Negative words: 

n't, even, get, wa, item, could, month, unit, would, buy, first, time,
week, back, bad, two, try, worked, doe, money, thing, support, tried,
poor, returned, return, warranty, waste, customer, refund, n't, even,
get, wa, item, could, month, unit, would, buy, first, time, week,
back, bad, two, try, worked, doe, money, thing, support, tried, poor,
returned, return, warranty, waste, customer, refund
